In [1]:
from rethon.ensemble_generation import EnsembleGenerator, SimpleEnsembleGenerator
from rethon.ensemble_generation import create_random_argument_list, standard_model_params_varied_alphas
from rethon.ensemble_generation import random_position_as_set, create_random_arguments2
from rethon.model import StandardPosition, DAGDialecticalStructure, StandardGlobalReflectiveEquilibrium
from os import getcwd
from multiprocessing import Process
import time

In [2]:
def experiment(options):
    arguments_list = options[0]
    n = options [1]
    coms_list = options [2]
    implementations = options [3]
    filename = options [4]
    
    start = time.perf_counter()

    model_params = [{
        'neighbourhood_depth': 1,
        'faithfulness_penalties': [0.0, 0.0, 1.0, 1.0],
        'account_penalties': [0.0, 0.3, 1.0, 1.0],
        'weights': {'account': 0.35, 'faithfulness': 0.1, 'systematicity': 0.55}
    }]
    ensemble_gen = SimpleEnsembleGenerator(arguments_list = arguments_list, 
                                     n_sentence_pool = n,
                                     initial_commitments_list = coms_list,
                                     model_parameters_list = model_params,
                                     implementations = implementations)


    # removing items we don't need
    ensemble_gen.remove_item('account_penalties')
    ensemble_gen.remove_item('faithfulness_penalties')
    ensemble_gen.remove_item('weight_account')
    ensemble_gen.remove_item('weight_systematicity')
    ensemble_gen.remove_item('weight_faithfulness')
    ensemble_gen.remove_item('n_random_choices')
    ensemble_gen.remove_item('n_init_coms_min_ax_base')
    ensemble_gen.remove_item('n_fixed_point_coms_min_ax_base')
    ensemble_gen.remove_item('fixed_point_coms_min_ax_bases')
    ensemble_gen.remove_item('init_coms_min_ax_bases')

    # defining the function for the global Optima
    def data_global_optima(ensemble_generator):
        re_model = ensemble_generator.reflective_equilibrium()
        #if re_model.model_name() == "StandardModel": fix for new version....
        if re_model.model_name() == "GlobalNumpyReflectiveEquilibrium":
            return re_model.global_optima(ensemble_generator.initial_commitments())

    ensemble_gen.add_item('global optima', data_global_optima)
    argument_size = len(arguments_list[0])
    #max_premises = 

    ensemble_gen.ensemble_items_to_csv(
                                  output_file_name = f'data_output_{filename}.csv',
                                  output_dir_name = getcwd(),
                                  archive = False, # save the csv as archived tar.gz
                                  save_preliminary_results = True, # will create preliminary csv-data sets 
                                  preliminary_results_interval = 50)
    finish = time.perf_counter()
    print(f'{filename}: {round(finish-start,2)}')
    return f'{filename}: {round(finish-start,2)}'

In [6]:
def createStruct(argument_size, max_premises, pool):
    # Sentence pool
    n= pool
    # Two random dialectical structures with 7 arguments each
    n_dialectical_structures = 50
    arguments_list = [create_random_arguments2(n_sentences=n, 
                                               n_arguments=argument_size,
                                               n_max_premises=max_premises) for i in range(n_dialectical_structures)]
    # Two random initial commitments
    n_positions = 2
    coms_list = [random_position_as_set(n_sentences = 2) for i in range(n_positions)]
    local_impl = [{'module_name': 'rethon.model',
               'position_class_name':'StandardPosition',
               'dialectical_structure_class_name': 'BDDDialecticalStructure',
               'reflective_equilibrium_class_name': 'StandardLocalReflectiveEquilibrium'
               }]
    global_impl = [{'module_name': 'rethon.model',
                'position_class_name':'StandardPosition',
                'dialectical_structure_class_name': 'BDDDialecticalStructure',
                'reflective_equilibrium_class_name': 'StandardGlobalReflectiveEquilibrium'}]
    return [[arguments_list, n, coms_list, local_impl, f"2commits_args-{argument_size}_premises-{max_premises}_pool-{n}_model-local"],[arguments_list, n, coms_list, global_impl, f"2commits_args-{argument_size}_premises-{max_premises}_pool-{n}_model-global"]]
            

args-7_premises-2_pool-5_model-local: 9.97
args-7_premises-2_pool-5_model-global: 10.48
args-7_premises-2_pool-6_model-local: 25.07
args-7_premises-2_pool-6_model-global: 27.31
args-7_premises-2_pool-7_model-global: 86.99
args-7_premises-2_pool-7_model-local: 96.81
args-7_premises-2_pool-8_model-global: 223.34
args-7_premises-2_pool-8_model-local: 333.25
args-7_premises-2_pool-9_model-global: 1117.32
args-7_premises-2_pool-9_model-local: 1449.93
args-7_premises-2_pool-10_model-global: 3779.62
args-7_premises-2_pool-10_model-local: 5655.59
args-7_premises-2_pool-11_model-global: 7325.45
args-7_premises-2_pool-11_model-local: 18764.94

BDD:
args-7_premises-2_pool-8_model-global: 648.03
args-7_premises-2_pool-8_model-local: 671.24
args-7_premises-2_pool-9_model-global: 2275.82
args-7_premises-2_pool-9_model-local: 2479.25


In [7]:
# print(experiment(createStruct(7,2,9)[0]))
# with concurrent.futures.ThreadPoolExecutor() as executor:
#    counter = []
#    for argsize in range(20):
#        counter.append(executor.map(experiment, createStruct(7,2,argsize+5)))

#    for count in counter:
#        for s in count:
#            print(s)
#options = createStruct(2,1)[0]
#experiment(options)

if __name__ == '__main__':
    for argsize in range(5, 11):
        for maxprem in range (1,3):
            options = createStruct(argsize,maxprem,argsize)
            p1 = Process(target=experiment, args=(options[0],))
            p2 = Process(target=experiment, args=(options[1],))
            p1.start()
            p2.start()
            if argsize % 4 == 0:
                p1.join()
                p2.join()

2commits_args-8_premises-1_pool-8_model-local: 472.5
2commits_args-8_premises-1_pool-8_model-global: 641.19
2commits_args-8_premises-2_pool-8_model-local: 569.83
2commits_args-8_premises-2_pool-8_model-global: 718.12
2commits_args-9_premises-1_pool-9_model-global: 3192.0
2commits_args-9_premises-1_pool-9_model-local: 3384.91
2commits_args-9_premises-2_pool-9_model-local: 3616.97
2commits_args-9_premises-2_pool-9_model-global: 3712.41
2commits_args-10_premises-1_pool-10_model-local: 7989.48
2commits_args-10_premises-1_pool-10_model-global: 9351.06
2commits_args-10_premises-2_pool-10_model-global: 10843.41
2commits_args-10_premises-2_pool-10_model-local: 11213.12


args-5_premises-1_pool-5_model-local: 38.95
args-5_premises-1_pool-5_model-global: 43.03
args-5_premises-2_pool-5_model-local: 44.93
args-5_premises-2_pool-5_model-global: 48.08
args-6_premises-1_pool-6_model-local: 81.13
args-6_premises-1_pool-6_model-global: 92.99
args-6_premises-2_pool-6_model-local: 97.75
args-6_premises-2_pool-6_model-global: 103.09
args-7_premises-1_pool-7_model-local: 159.94
args-7_premises-2_pool-7_model-local: 189.54
args-7_premises-1_pool-7_model-global: 198.68
args-7_premises-2_pool-7_model-global: 210.99
args-8_premises-1_pool-8_model-local: 439.5
args-8_premises-1_pool-8_model-global: 607.37
args-8_premises-2_pool-8_model-local: 393.75
args-8_premises-2_pool-8_model-global: 462.97
args-9_premises-1_pool-9_model-local: 1460.39
args-9_premises-1_pool-9_model-global: 1825.38
args-9_premises-2_pool-9_model-global: 2048.57
args-9_premises-2_pool-9_model-local: 2270.28
args-10_premises-1_pool-10_model-local: 2672.82
args-10_premises-1_pool-10_model-global: 4291.83
args-10_premises-2_pool-10_model-local: 4686.19
args-10_premises-2_pool-10_model-global: 5345.92


args-5_premises-1_pool-5_model-local: 5.36
args-5_premises-1_pool-5_model-global: 7.43
args-6_premises-1_pool-6_model-local: 13.34
args-6_premises-1_pool-6_model-global: 19.23
args-7_premises-1_pool-7_model-local: 37.35
args-7_premises-1_pool-7_model-global: 52.98
args-7_premises-2_pool-5_model-local: 4.39
args-7_premises-2_pool-5_model-global: 7.11
args-7_premises-2_pool-6_model-local: 20.53
args-7_premises-2_pool-6_model-global: 27.16
args-7_premises-2_pool-7_model-local: 64.7
args-7_premises-2_pool-7_model-global: 80.93
args-7_premises-2_pool-8_model-local: 220.38
args-7_premises-2_pool-8_model-global: 281.46
args-7_premises-2_pool-9_model-global: 743.79
args-7_premises-2_pool-9_model-local: 868.99
args-7_premises-2_pool-10_model-global: 3112.25
args-7_premises-2_pool-10_model-local: 4773.51
args-7_premises-2_pool-11_model-global: 14401.48
args-7_premises-2_pool-11_model-local: 22881.2
args-5_premises-1_pool-5_model-local: 41.78
args-5_premises-1_pool-5_model-global: 43.42
args-5_premises-2_pool-5_model-global: 46.76
args-5_premises-2_pool-5_model-local: 47.3
args-6_premises-1_pool-6_model-local: 83.44
args-6_premises-1_pool-6_model-global: 91.57
args-6_premises-2_pool-6_model-local: 96.54
args-6_premises-2_pool-6_model-global: 97.9
args-7_premises-2_pool-7_model-local: 175.2
args-7_premises-1_pool-7_model-local: 180.95
args-7_premises-2_pool-7_model-global: 189.8
args-7_premises-1_pool-7_model-global: 202.9
args-8_premises-1_pool-8_model-local: 377.07
args-8_premises-1_pool-8_model-global: 479.77
args-8_premises-2_pool-8_model-local: 429.9
args-8_premises-2_pool-8_model-global: 487.21
args-9_premises-1_pool-9_model-global: 1641.61
args-9_premises-1_pool-9_model-local: 1040.25
args-9_premises-1_pool-9_model-global: 1353.08
args-9_premises-2_pool-9_model-global: 1860.98
args-9_premises-2_pool-9_model-local: 2063.1
args-10_premises-2_pool-10_model-local: 4188.06
args-10_premises-2_pool-10_model-global: 4199.59
args-10_premises-1_pool-10_model-global: 4559.8
args-9_premises-1_pool-9_model-local: 1040.25
args-9_premises-1_pool-9_model-global: 1353.08
